In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
import pandas as pd

load_dotenv()
user_name = os.environ.get("sql_user")
password = os.environ.get("sql_password")
db_name = os.environ.get("sql_db_name")
sql_ip = os.environ.get("sql_ip")
sql_port = os.environ.get("sql_port")

db_url = f"mariadb+mariadbconnector://{user_name}:{password}@{sql_ip}:{sql_port}/"f"{db_name}"

engine = create_engine(db_url, pool_pre_ping=True, pool_recycle=3600)
conn = engine.connect()
meta = MetaData()
meta.reflect(conn)

In [41]:
# Down-sampling (took about 10 sec for 3000-4000)
original_table = 'balance_sheet_annual'
sql = f'SELECT * FROM {original_table}'
result = engine.execute(sql).fetchall()

In [42]:
df_result = pd.DataFrame(result)

In [43]:
df_result.set_index('date', inplace=True)
df_result.index = pd.DatetimeIndex(df_result.index)


In [44]:
symbols = df_result['symbol'].unique()
symbol = 'A'

In [45]:
df_result_symbol = df_result[df_result['symbol'] == symbol]

In [47]:
from pandas.api.types import is_numeric_dtype

numeric_columns = [is_numeric_dtype(df_result_symbol['symbol'])]

[False]

In [51]:
numeric_columns = [column for column in df_result_symbol.columns if is_numeric_dtype(df_result_symbol[column])]
df_change_symbol = pd.concat(
    [df_result_symbol['symbol']] + [df_result_symbol[numeric_column].pct_change() for numeric_column in
                                    numeric_columns], axis=1).dropna()
df_change_symbol.index = df_change_symbol.index.strftime("%Y-%m-%d")
df_change_symbol.reset_index().to_dict('records')



[]

In [60]:
pd.concat(
    [df_result_symbol['symbol']] + [df_result_symbol[numeric_column].pct_change() for numeric_column in
                                    numeric_columns], axis=1).dropna(how="all",axis=0,thresh=2)

,symbol,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,...,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalEquity,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt
date,,,,,,,,,,,,,,,,,,,,,
1999-10-31,A,NaN,NaN,NaN,0.345679,0.009428,0.077333,0.150569,-0.063471,NaN,...,0.106777,0.113345,0.119126,0.091638,NaN,0.119126,0.091638,NaN,NaN,NaN
2000-10-31,A,inf,NaN,inf,0.346177,0.236157,0.497525,0.598361,0.255227,NaN,...,-0.018062,0.191926,0.556771,0.547575,NaN,0.556771,0.547575,NaN,inf,-inf
2001-10-31,A,0.174699,NaN,0.174699,-0.556111,-0.195359,0.919008,-0.151370,0.061459,NaN,...,0.021534,0.039810,0.074834,-0.052107,NaN,0.074834,-0.052107,NaN,-0.945455,0.313770
2002-10-31,A,0.576068,NaN,0.576068,0.145343,-0.205902,-0.368648,0.016879,-0.145563,NaN,...,-0.934563,-0.304000,-0.182364,0.027173,NaN,-0.182364,0.027173,NaN,190.666667,-0.403780
2003-10-31,A,-0.128525,NaN,-0.128525,-0.029491,-0.159628,-0.725784,-0.203074,-0.083597,NaN,...,-0.959732,0.022989,-0.389669,-0.232354,NaN,-0.389669,-0.232354,NaN,0.000000,-0.341499
2004-10-31,A,0.440572,NaN,0.440572,-0.038674,0.031156,-0.044776,0.176909,-0.130615,NaN,...,-30.833333,0.042335,0.263810,0.120534,NaN,0.263810,0.120534,NaN,0.000000,1.549234
2005-10-31,A,-0.038445,inf,-0.027646,-0.278736,-0.296296,2.755208,-0.028403,-0.306041,NaN,...,-1.273743,0.075650,0.143458,-0.043226,NaN,0.143458,-0.043226,NaN,-1.000000,0.910730
2006-10-31,A,0.016173,-1.000000,0.004887,-0.081009,-0.131579,-0.477115,-0.109962,-0.112257,inf,...,-1.591837,-0.627774,-0.106101,0.089913,inf,-0.103406,0.091542,NaN,inf,-0.657682
2007-10-31,A,-0.192750,NaN,-0.192750,0.062139,0.025518,0.238727,-0.072511,0.033548,0.911688,...,13.068966,-0.688462,-0.113487,0.026638,-1.000000,-0.116152,0.025105,NaN,0.391333,-1.342520


In [58]:
df_result_symbol['inventory'].pct_change()

date
1998-10-31         NaN
1999-10-31    0.009428
2000-10-31    0.236157
2001-10-31   -0.195359
2002-10-31   -0.205902
2003-10-31   -0.159628
2004-10-31    0.031156
2005-10-31   -0.296296
2006-10-31   -0.131579
2007-10-31    0.025518
2008-10-31    0.004666
2009-10-31   -0.145511
2010-10-31    0.297101
2011-10-31    0.254190
2012-10-31    0.129176
2013-10-31    0.051282
2014-10-31    0.005629
2015-10-31   -0.495336
2016-10-31   -0.014787
2017-10-31    0.078799
2018-10-31    0.109565
2019-10-31    0.064263
2020-10-31    0.060383
2021-10-31    0.152778
Name: inventory, dtype: float64

In [84]:
symbols = df_result['symbol'].unique()
symbol = 'A'
resample_period = '6m'
# for symbol in symbols:
df_result_symbol = df_result[df_result['symbol'] == symbol].resample(resample_period).mean()
df_result_symbol.dropna(inplace=True)
result_dict_list = [{'symbol': symbol, 'date': date, 'adjClose': adjClose, 'changePercent': changePercent} for
                    date, adjClose, changePercent in
                    zip(list(df_result_symbol.index.strftime("%Y-%m-%d")), df_result_symbol['adjClose'],
                        df_result_symbol['changePercent'])]

In [54]:
from core.db_interface import TableName

table_name = "weekly_price"


In [91]:
df_result_symbol.reset_index().to_dict('records')

[{'date': Timestamp('1999-11-30 00:00:00'),
  'adjClose': 25.8214,
  'changePercent': -15.979},
 {'date': Timestamp('2000-05-31 00:00:00'),
  'adjClose': 50.98834047619047,
  'changePercent': -13.771642857142856},
 {'date': Timestamp('2000-11-30 00:00:00'),
  'adjClose': 33.75073720930232,
  'changePercent': -15.00706976744186},
 {'date': Timestamp('2001-05-31 00:00:00'),
  'adjClose': 27.09636097560976,
  'changePercent': -14.392658536585367},
 {'date': Timestamp('2001-11-30 00:00:00'),
  'adjClose': 16.465260975609755,
  'changePercent': -13.020365853658538},
 {'date': Timestamp('2002-05-31 00:00:00'),
  'adjClose': 18.756057142857145,
  'changePercent': -14.48995238095238},
 {'date': Timestamp('2002-11-30 00:00:00'),
  'adjClose': 10.597185476190477,
  'changePercent': -13.418690476190477},
 {'date': Timestamp('2003-05-31 00:00:00'),
  'adjClose': 9.587880975609757,
  'changePercent': -14.323243902439025},
 {'date': Timestamp('2003-11-30 00:00:00'),
  'adjClose': 14.100976744186045,

In [58]:
columns = [Column(column.name, column.type, primary_key=column.primary_key, index=column.index) for column in
           daily_price_table.c]
table = Table(table_name, meta, *columns, extend_existing=True)

In [60]:
from sqlalchemy.dialects.mysql import insert

stmt = insert(table).values(result_dict_list)
update_dict = {x.name: x for x in stmt.inserted}
on_duplicate_key_stmt = stmt.on_duplicate_key_update(update_dict)
conn.execute(statement=on_duplicate_key_stmt)

In [71]:
sql_query = f"SELECT date FROM {table_name} where symbol = '{symbol}' order by date desc limit 1;"
result = conn.execute(sql_query).fetchall()
if result:
    print(result[0][0])



2022-09-18


In [72]:
type(result[0][0])

datetime.date

In [73]:
a_dict = {'a': 1, 'b': 2}
None in a_dict.keys()

False

In [78]:
from datetime import datetime

a_date = datetime.strptime('2019-01-02', '%Y-%m-%d').date()
a_date

datetime.date(2019, 1, 2)

In [83]:
import datetime

a_date - a_date < datetime.timedelta(days='1w')

TypeError: unsupported type for timedelta days component: str